In [ ]:
%matplotlib ipympl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pymoo.indicators.hv import HV

# NSGA-II

In [ ]:
methods = [
    "single-cutcat-unif",
    "multi-cutcat-unif",
    "single-cutcat-tour",
    "multi-cutcat-tour"
]


def nsga2_plot3d(df, name, e=40, a=-146, v=None):
    fig = plt.figure()
    axes = plt.axes(projection='3d')
    # if v != None:
    #     x, y, z = v
    #     axes.scatter3D([x], [y], [z], label="Reference point")
    for m in methods:
        to_use = df[df["method"] == m]

        axes.scatter3D(to_use['build_time'], to_use['search_time'], to_use['recall'], label=m)
        axes.set_xlabel('build_time')
        axes.set_ylabel('search_time')
        axes.set_zlabel('recall')

        axes.view_init(elev=e, azim=a)

    plt.legend()
    plt.savefig(f'../images/report/{name}.png')
    plt.show()


def nsga2_plot3d2(to_use, name, e=40, a=-146, v=None, set_limits=True):
    fig = plt.figure()
    axes = plt.axes(projection='3d')

    axes.scatter3D(to_use['build_time'], to_use['search_time'], to_use['recall'])
    axes.set_xlabel('build_time')
    axes.set_ylabel('search_time')
    axes.set_zlabel('recall')

    if set_limits:
        axes.set_xlim(0, 1)
        axes.set_ylim(0, 1)
        axes.set_zlim(0, 1)

    axes.view_init(elev=e, azim=a)
    plt.savefig(f'../images/report/{name}.png', bbox_inches="tight")
    plt.show()

In [ ]:
# non_scaling = pd.read_csv("result-non-scaling.csv")
non_scaling_frontier = pd.read_csv("result-non-scaling-frontier.csv")

# scaling = pd.read_csv("result-scaling.csv")
scaling_frontier = pd.read_csv("result-scaling-frontier.csv")

In [ ]:
scaling_frontier = scaling_frontier[scaling_frontier["generation"] == 10]
non_scaling_frontier = non_scaling_frontier[non_scaling_frontier["generation"] == 10]

## Non-scaling

In [ ]:
rp = [7, 0.01, 1]
ind = HV(ref_point=rp)
nsga2_plot3d(non_scaling_frontier, "non-scaling-frontier", 37, -162)

In [ ]:
nsga2_plot3d2(non_scaling_frontier[non_scaling_frontier["method"] == "single-cutcat-unif"],
              "non-scaling-frontier-single-unif", 37, -162, rp, set_limits=False)

In [ ]:
nsga2_plot3d2(non_scaling_frontier[non_scaling_frontier["method"] == "multi-cutcat-unif"],
              "non-scaling-frontier-multi-unif", 37, -162, rp, set_limits=False)

In [ ]:
nsga2_plot3d2(non_scaling_frontier[non_scaling_frontier["method"] == "single-cutcat-tour"],
              "non-scaling-frontier-single-tour", 37, -162, rp, set_limits=False)

In [ ]:
nsga2_plot3d2(non_scaling_frontier[non_scaling_frontier["method"] == "multi-cutcat-tour"],
              "non-scaling-frontier-multi-tour", 37, -162, rp, set_limits=False)

In [ ]:
for m in methods:
    to_use = non_scaling_frontier[non_scaling_frontier["method"] == m]
    print(f"HV of {m}:",
          ind(np.array([to_use["build_time"], to_use["search_time"], 1 - to_use["recall"]]).transpose()))

In [ ]:
def compute_row_hv(row):
    b = row["build_time"]
    s = row["search_time"]
    r = row["recall"]
    return ind(np.array([b, s, 1 - r]))


non_scaling_frontier["hv"] = non_scaling_frontier.apply(compute_row_hv, axis=1)
sorted_nsf = non_scaling_frontier.sort_values(by="hv", ascending=False)[:10]
print(sorted_nsf.drop(["method", "generation", "memory", "trial"], axis=1).to_latex())
sorted_nsf

## Scaling

In [ ]:
rp = [1, 1, 1]
nsga2_plot3d(scaling_frontier, "scaling-frontier", 37, -162, rp)

In [ ]:
nsga2_plot3d2(scaling_frontier[scaling_frontier["method"] == "single-cutcat-unif"], "scaling-frontier-single-unif", 37,
              -162, rp)

In [ ]:
nsga2_plot3d2(scaling_frontier[scaling_frontier["method"] == "multi-cutcat-unif"], "scaling-frontier-multi-unif", 37,
              -162, rp)

In [ ]:
nsga2_plot3d2(scaling_frontier[scaling_frontier["method"] == "single-cutcat-tour"], "scaling-frontier-single-tour", 37,
              -162, rp)

In [ ]:
nsga2_plot3d2(scaling_frontier[scaling_frontier["method"] == "multi-cutcat-tour"], "scaling-frontier-multi-tour", 37,
              -162, rp)

In [ ]:
for m in methods:
    to_use = scaling_frontier[scaling_frontier["method"] == m]
    ind = HV(ref_point=rp)
    print(f"HV of {m}:",
          ind(np.array([to_use["build_time"], to_use["search_time"], 1 - to_use["recall"]]).transpose()))

In [ ]:
scaling_frontier["hv"] = scaling_frontier.apply(compute_row_hv, axis=1)
sorted_sf = scaling_frontier.sort_values(by="hv", ascending=False)[:10]
print(sorted_sf.drop(["method", "generation", "memory", "trial"], axis=1).to_latex())
sorted_sf.drop(["method", "generation", "memory", "trial"], axis=1)

# Weighted Sum with BO

In [ ]:
from algorithms.moo import BuildParams
from algorithms.nsga2 import fast_non_dominated_sort


def form_individual(row):
    bp = BuildParams([row["M"], row["C"], row["S"], row["alpha"]])
    bp.function_values = np.array([row["build_time"], row["search_time"], 0, 1 - row["recall"]])
    return bp


def inds_to_dataframe(individuals):
    form_individuals = [[
        individual.v[0], individual.v[1], individual.v[2], individual.v[3],  # M, C, S, alpha
        individual.function_values[0], individual.function_values[1], 1 - individual.function_values[3],
        # build_time, search_time, recall
    ] for individual in individuals]
    columns = ["M", "C", "S", "alpha", "build_time", "search_time", "recall"]
    # columns = ["$M$", "$C$", "$S$", "$\\alpha$", "$f_c$", "$f_s$", "$f_r$"]
    return pd.DataFrame(form_individuals, columns=columns)

In [ ]:
def get_frontier_df(path):
    bo_results = pd.read_csv(path).sort_values(by="ws", ascending=False)
    individuals = [form_individual(row) for _, row in bo_results.iterrows()]
    first_frontier = fast_non_dominated_sort(individuals)[0]
    return inds_to_dataframe(first_frontier)


paths = [f"bo-small-{w}-unscaled.csv" for w in ["111", "211", "122", "123"]]
bo_frontiers = [get_frontier_df(path) for path in paths]

In [ ]:
# fig = plt.figure()
# axes = plt.axes(projection='3d')
# axes.set_xlim(0, 1)
# axes.set_ylim(0, 1)
# axes.set_zlim(0, 1)
# axes.view_init(elev=20, azim=-162)
# 
# weights = ["111", "211", "122", "123"]
# for i in range(4):
#     axes.scatter3D(bo_frontiers[i]['build_time'],
#                    bo_frontiers[i]['search_time'],
#                    bo_frontiers[i]['recall'],
#                    label=f"weight: {','.join(weights[i])}")
# 
# axes.set_xlabel('build_time')
# axes.set_ylabel('search_time')
# axes.set_zlabel('recall')
# 
# plt.legend()
# plt.savefig(f'../images/report/bo-frontiers-tgt.png', bbox_inches="tight")
# plt.show()

In [ ]:
# nsga2_plot3d2(bo_frontiers[0], "bo-frontier111", 25, -162)

In [ ]:
# nsga2_plot3d2(bo_frontiers[1], "bo-frontier211", 25, -162)

In [ ]:
# nsga2_plot3d2(bo_frontiers[2], "bo-frontier122", 25, -162)

In [ ]:
# nsga2_plot3d2(bo_frontiers[3], "bo-frontier123", 25, -162)

In [ ]:
weights = ["111", "211", "122", "123"]
for i in range(4):
    ind = HV(ref_point=[1, 1, 1])
    print(f"HV of {weights[i]}:",
          ind(np.array([bo_frontiers[i]["build_time"],
                        bo_frontiers[i]["search_time"],
                        1 - bo_frontiers[i]["recall"]]).transpose()))

In [ ]:
all_bo_frontiers = pd.concat(bo_frontiers)
all_bo_frontiers["hv"] = all_bo_frontiers.apply(compute_row_hv, axis=1)
sorted_bo = all_bo_frontiers.sort_values(by="hv", ascending=False)[:10]
print(sorted_bo.to_latex())
sorted_bo

# Comparing the two

In [ ]:
all_bo_frontiers["method"] = "bo"
scaling_frontier["method"] = "nsga2"
all_frontiers = pd.concat([all_bo_frontiers, scaling_frontier]).reset_index(drop=True).dropna(axis=1)
all_frontiers

In [ ]:
bo_top10_fc = [0.056906, 0.058899, 0.059805, 0.124925, 0.127623, 0.128544, 0.128413, 0.128873, 0.125418, 0.129775]
bo_top10_fs = [0.000900, 0.000955, 0.000902, 0.003450, 0.001628, 0.001415, 0.002333, 0.002133, 0.000846, 0.001709]
bo_top10_fr = [0.968400, 0.968400, 0.966300, 1, 1, 0.9999, 1, 1, 0.9944, 1]
bo_top10_hv = [0.912471, 0.910492, 0.907691, 0.872056, 0.870957, 0.870135, 0.869553, 0.869269, 0.868949, 0.868737]

n2_top10_fc = [0.018249, 0.030292, 0.001252, 0.001252, 0.001252, 0.001252, 0.001252, 0.033138, 0.033138, 0.033138]
n2_top10_fs = [0.004453, 0.00689, 0.039502, 0.039502, 0.039502, 0.039502, 0.039502, 0.007289, 0.007289, 0.007289]
n2_top10_fr = [0.9993, 0.9959, 0.9994, 0.9994, 0.9994, 0.9994, 0.9994, 0.9954, 0.9954, 0.9954]
n2_top10_hv = [0.976695, 0.959078, 0.958719, 0.958719, 0.958719, 0.958719, 0.958719, 0.955399, 0.955399, 0.955399]

print(
    f"BO: {np.mean(bo_top10_fc):.6f} & {np.mean(bo_top10_fs):.6f} & {np.mean(bo_top10_fr):.6f} & {np.mean(bo_top10_hv):.6f}")
print(
    f"NSGA-II: {np.mean(n2_top10_fc):.6f} & {np.mean(n2_top10_fs):.6f} & {np.mean(n2_top10_fr):.6f} & {np.mean(n2_top10_hv):.6f}")

In [ ]:
fig = plt.figure()
axes = plt.axes(projection='3d')
axes.set_xlim(0, 1)
axes.set_ylim(0, 1)
axes.set_zlim(0, 1)
axes.view_init(elev=20, azim=-162)

axes.scatter3D(all_frontiers[all_frontiers["method"] == "bo"]['build_time'],
               all_frontiers[all_frontiers["method"] == "bo"]['search_time'],
               all_frontiers[all_frontiers["method"] == "bo"]['recall'],
               label=f"BO")
axes.scatter3D(all_frontiers[all_frontiers["method"] == "nsga2"]['build_time'],
               all_frontiers[all_frontiers["method"] == "nsga2"]['search_time'],
               all_frontiers[all_frontiers["method"] == "nsga2"]['recall'],
               label=f"NSGA-II")

axes.set_xlabel('build_time')
axes.set_ylabel('search_time')
axes.set_zlabel('recall')

plt.legend()
plt.savefig(f'../images/report/bo-vs-nsga2.png', bbox_inches="tight")
plt.show()